In [37]:
from itertools import chain
from lnn import Predicate, Variable, Exists, Implies, And, ForAll, Model, Fact, World

## Domain specification

In [38]:
# entities and properties names
entities = ["rectangle", "square", "rhombus", "parallelogram"]
properties = ["foursided", "equal_sides", "right_angles"]

# implications
entities_relationships = [
    ("square", "rectangle"),
    ("square", "rhombus"),
    ("rectangle", "parallelogram"),
    ("rhombus", "parallelogram"),
]

# implications
entities_properties = [
    ("parallelogram", "foursided"),
    ("rectangle", "right_angles"),
    ("rhombus", "equal_sides"),
]

# information that object "s1" is a square etc.
facts = {
    "square": ["s1", "s2"],
    "rectangle": ["r1"],
    "rhombus": ["rh1"],
    "parallelogram": ["p1"],
}

## Model definition and domain parsing

In [ ]:
model = Model()

x = Variable("x")

# parsing predicates
for predicate in chain(entities, properties):
    model[predicate] = Predicate(name=predicate)

# parsing axioms in Implication and ForAll shape
axioms = [
    ForAll(
        x,
        Implies(model[p1](x), model[p2](x), name=f"{p1}_{p2}"),
        name=f"all_{p1}_{p2}",
        world=World.AXIOM,
    )
    for p1, p2 in chain(entities_relationships, entities_properties)
]

model.add_formulae(*axioms)

# parsing facts
model.add_facts(
    {entity: {name: Fact.TRUE for name in names} for entity, names in facts.items()}
)

## Question definition

In [ ]:
queries = [
    Exists(x, model["foursided"](x), name="foursided_objects"),
    Exists(x, model["equal_sides"](x), name="equal_sides_objects"),
    Exists(x, model["right_angles"](x), name="right_angles_objects"),
    Exists(
        x,
        And(model["right_angles"](x), model["equal_sides"](x)),
        name="right_angles_and_equal_length_sides",
    ),
]

model.add_formulae(*queries)

## Infer from model and answer questions

In [39]:
steps, facts_inferred = model.infer()

for query in queries:
    print(f"Query {query.name} result:")
    print(model[query.name].true_groundings)

Query foursided_objects result:
{'p1', 'r1', 'rh1', 's1', 's2'}
Query equal_sides_objects result:
{'s2', 'rh1', 's1'}
Query right_angles_objects result:
{'s2', 'r1', 's1'}
Query right_angles_and_equal_length_sides result:
{'s2', 's1'}
